## get_progeny_benchmark
The purpose of this notebook is to speed test getting the list of all progeny (aka descendents) of a given brain region using two different methods. The first method is the old way of looping through the whole graph recursively until hitting the parent node. The second is constructing the relationships between nodes when making the graph. whole graph first and using the node relationships of the graph to get the progeny.

Method 2 turns out to be MUCH faster. 

In [40]:
import json
import graphviz
import csv
import numpy as np
%matplotlib inline

In [42]:
ontology_file = 'allen_ontology_16bit.json'
with open(ontology_file) as json_file:
    data = json.load(json_file)

## Method 1: Graphviz API

In [45]:
def make_graph(dic,graph):
    """ Make a edge-unweighted directed graph from a dictionary
    Representing a brain ontology using the graphviz API
    """
    name = dic.get('name')
    children = dic.get('children')
    for child in children:
        child_name = child.get('name')
        graph.edge(name,child_name)
        make_graph(child,graph)
    return 

In [46]:
G = graphviz.Digraph()
make_graph(data,G)

In [47]:
# Now write the function to get all progeny of an input nodename
def get_progeny(dic,input_nodename,progeny_list):
    """ 
    """
    name = dic.get('name')
    children = dic.get('children')
    if name == input_nodename:
        for child in children: # child is a dict
            child_name = child.get('name')
            progeny_list.append(child_name)
            get_progeny(child,input_nodename=child_name,progeny_list=progeny_list)
        return
    
    for child in children:
        child_name = child.get('name')
        get_progeny(child,input_nodename=input_nodename,progeny_list=progeny_list)
    return 

## Method 2: Make a graph object composed of nodes

In [51]:
class Node():
    """ The elements in the graph"""
    def __init__(self,name):
        self.name = name
        self.parent = None
        self.children = [] # list of Node objects
        self.level = None
    def __repr__(self):
        return str(self.name)

In [52]:
class Graph():
    """ The ontology graph """
    def __init__(self):
        self.nodes = {} # map name to Node object
        self.html_str = ''
        self.level = 0
        self.last_checked_nodename = None
    
    def make_graph(self,dic):
        name = dic.get('name')
        if name not in self.nodes:
            node = Node(name)
            self.nodes[name] = node
        else:
            node = self.nodes[name]
        if name == 'root':
            node.level = 0
        children = dic.get('children')
        self.level += 1
        for child in children: # child is a dict
            child_name = child.get('name')
            child_node = Node(child_name)
            child_node.parent = node
            child_node.level = self.level 
            self.nodes[child_name] = child_node
            node.children.append(child_node)
            self.make_graph(child)
        self.level -= 1
    
    def print_branch(self,nodename,level=0,stoplevel=2):
        """ 
        ---PURPOSE---
        Print out the branch of this ontology, 
        starting with a parent node name
        ---INPUT---
        nodename     The parent node you want to start at
        level        An internal variable. Do not modify.
        stoplevel    The number of levels down from the parent 
                     node that you want to print. 
                     Use -1 to print the entire branch out
        """
        if stoplevel == -1:
            pass
        elif level > stoplevel:
            return
        
        print("\t"*level,level,nodename)
        for child in self.nodes[nodename].children:
            self.print_branch(child.name,level+1,stoplevel=stoplevel)
        level-=1
    
    def visualize_graph(self,nodename,stoplevel=-1):
        """ 
        ---PURPOSE---
        Visualize a graph starting at a given node
        and stopping at a certain number of levels down 
        the ontology.
        ---INPUT---
        nodename     The parent node you want to start at
        stoplevel    The number of levels down from the parent 
                     node that you want to print. -1 shows all
        """
        graph = graphviz.Digraph()
        self.visualize_graph_helper(nodename=nodename,graph=graph,level=0,stoplevel=stoplevel)
        return graph
    
    def visualize_graph_helper(self,nodename,graph,level,stoplevel):
        if stoplevel == -1:
            pass
        elif level >= stoplevel:
            return
        graph.node(nodename)
        for child in self.nodes[nodename].children:
            graph.edge(nodename,child.name)
            self.visualize_graph_helper(child.name,graph=graph,level=level+1,stoplevel=stoplevel)
        level-=1
        return
    
    def make_html_ontology(self):
        """ 
        ---PURPOSE---
        Print out the entire ontology
        in html format with regions
        that are included in the dataframe in bold
        ---INPUT---
        """
        # reset html_str
        self.html_str = ''
        self.html_helper(nodename='root')
        return self.html_str

    def html_helper(self,nodename):
        node = self.nodes[nodename]
        level = node.level
        
        if nodename == 'root':
            previous_level = -1
        else:
            previous_node = self.nodes[self.last_checked_nodename]
            previous_level = previous_node.level
        
        if level > previous_level:
            self.html_str += '<ul style="list-style-type: none;">'
        elif level == previous_level:
            # dont need to do anything
            pass
        elif level < previous_level:
            # then we need to end unordered lists
            # the number of which we need to end depends on the difference between
            # current and previous level
            level_diff = previous_level-level
            self.html_str += '</ul>'*level_diff
        # Add the actual text to the html string
        # use boldface if nodename is in the volume and therefore a dataframe column
        if nodename in main_df.columns:
            self.html_str += '<li>' + ' '.join([str(level),f'<b>{nodename}</b>','</li>'])
        else:
            self.html_str += '<li>' + ' '.join([str(level),nodename,'</li>'])
        self.last_checked_nodename = nodename
        for child in self.nodes[nodename].children:
            self.html_helper(child.name,)
        return
    
    def get_progeny(self,nodename):
        """ 
        ---PURPOSE---
        Return a list of progeny of the the given nodename
        ---INPUT---
        nodename     The parent node whose descendents you want
        """
        progeny_list = []
        
        self.get_progeny_helper(nodename,progeny_list)
        return progeny_list
    
    def get_progeny_helper(self,nodename,progeny_list):
        for child in self.nodes[nodename].children:
            progeny_list.append(child.name)
            self.get_progeny_helper(child.name,progeny_list)
        return

In [53]:
graph = Graph()
graph.make_graph(data)

## Compare methods

In [69]:
parent_regions_to_test = ['Thalamus','Striatum ventral region','Hindbrain','Basic cell groups and regions']

In [78]:
for parent_region in parent_regions_to_test:
    print(parent_region)
    print("Method 1:")
    progeny_list_method1 = []
    %time get_progeny(data,input_nodename='Thalamus',progeny_list=progeny_list_method1)
    print("Method 2:")
    %time progeny_list_method2 = graph.get_progeny('Thalamus')
    assert progeny_list_method2 == progeny_list_method1
    print()
    

Thalamus
Method 1:
CPU times: user 2.25 ms, sys: 0 ns, total: 2.25 ms
Wall time: 2.26 ms
Method 2:
CPU times: user 122 µs, sys: 0 ns, total: 122 µs
Wall time: 127 µs

Striatum ventral region
Method 1:
CPU times: user 1.77 ms, sys: 0 ns, total: 1.77 ms
Wall time: 1.78 ms
Method 2:
CPU times: user 98 µs, sys: 0 ns, total: 98 µs
Wall time: 102 µs

Hindbrain
Method 1:
CPU times: user 1.6 ms, sys: 31 µs, total: 1.63 ms
Wall time: 1.64 ms
Method 2:
CPU times: user 71 µs, sys: 0 ns, total: 71 µs
Wall time: 74.6 µs

Basic cell groups and regions
Method 1:
CPU times: user 1.25 ms, sys: 0 ns, total: 1.25 ms
Wall time: 1.26 ms
Method 2:
CPU times: user 64 µs, sys: 0 ns, total: 64 µs
Wall time: 66.5 µs



# Summary
Method 2 is much faster, especially for small brain areas. Method 1 takes approximately constant time for any region because it needs to loop through the whole brain, whereas Method 2 only searches the parent region down.  Both take up approximately the same memory since they must construct the whole graph ahead of time, which is small for these brain atlases. Another advantage of Method 2 is that it does not rely on an external library (like graphviz for Method 1). Another advantage of Method 2 is that it creates and returns the progeny list internally. Method 1 requires a list to be created first and then appends to that list in place, which is less clean.